In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
texts = pd.read_csv("/content/+С2_RuFoLa_corpus - +С2_RuFoLa_corpus.csv")
texts

,id,level,source,text,source.1,keys
0,1,1,jilibili_1,Это мой друг. Это моя подруга. Это мой дом. Эт...,NaN,NaN
1,2,1,jilibili_1,Это я. Меня зовут Анна. Я студентка.Это мой от...,NaN,NaN
2,3,1,jilibili_1,Это Иван Петрович Сидоров. А это его ученики. ...,NaN,NaN
3,4,1,jilibili_1,Мы изучаем русский ЯЗЫК.Это наша аудитория. Се...,NaN,NaN
4,5,1,jilibili_1,Наш город.Петербург.Он не очень старый. Здесь ...,NaN,NaN
...,...,...,...,...,...,...
2082,2098,6,youlang,"Конечно же, не стоит забывать об элементарных ...",NaN,NaN
2083,2099,6,youlang,Светлана Комарова: кандидат психологических на...,NaN,NaN
2084,2100,6,youlang,"Наталья Искра: кандидат психологических наук, ...",NaN,NaN
2085,2101,6,youlang,На принятие решения о создании семьи в первую ...,NaN,NaN


In [ ]:
texts.shape

(2087, 6)

In [ ]:
counts = texts['level'].value_counts()
counts

3    586
5    422
2    375
1    356
4    305
6     43
Name: level, dtype: int64

In [ ]:
import torch.nn as nn

import random
import torch


def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


set_random_seed(42)

In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.2 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")          # сюда можно подставить другую
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")

Функция получения эмбеддингов:

In [ ]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

print(embed_bert_cls('привет мир', model, tokenizer).shape)

(312,)


In [ ]:
texts.shape[0]

2087

Получаем эмбеддинги:

In [ ]:
set_random_seed(42)

from tqdm.notebook import tqdm
from torch.nn.utils.rnn import pad_sequence

features_po_tiny = []

for index, row in tqdm(texts.iterrows(), total=texts.shape[0]):
    emb = embed_bert_cls(row["text"], model, tokenizer)
    features_po_tiny.append(emb)


  0%|          | 0/2087 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(features_po_tiny)
# df.to_csv("/content/features_plus_c2_tiny.csv", index=True)

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,302,303,304,305,306,307,308,309,310,311
0,0.013404,0.081277,-0.002718,0.006781,-0.021074,-0.001101,0.061832,0.013175,0.007073,0.062094,...,0.053784,0.051911,0.011225,0.030491,-0.012882,0.033562,0.110991,-0.023343,0.023444,-0.097022
1,-0.005927,0.059065,-0.040431,-0.006000,-0.020420,-0.034996,0.031260,0.016466,-0.008110,0.075843,...,0.013393,-0.009853,-0.006460,0.066707,-0.007403,0.025527,0.126925,-0.032500,-0.011747,-0.102165
2,0.062544,0.024377,-0.053184,-0.010623,0.000207,-0.063394,0.034824,0.014547,-0.007983,0.044286,...,0.001290,-0.057028,-0.022071,0.068926,-0.015586,0.045683,0.069069,-0.049771,-0.002949,-0.024824
3,0.088431,0.049204,-0.041520,-0.021549,0.002006,-0.061517,0.026090,-0.013590,0.040709,0.055408,...,-0.014529,-0.058937,-0.037493,0.057122,-0.024982,0.010939,-0.003627,-0.064586,-0.026422,-0.022364
4,0.139185,0.043936,-0.022766,0.003944,-0.007901,-0.041158,0.024030,-0.018649,0.038063,0.045925,...,-0.028922,0.086758,-0.050221,0.048976,-0.052322,-0.012005,-0.079502,-0.043892,0.054148,-0.025723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2082,0.074340,0.051397,0.008666,-0.025970,0.037153,-0.026889,0.010542,-0.059984,-0.009359,0.033215,...,-0.005907,-0.106972,-0.007764,0.037729,-0.032021,0.021655,-0.048305,-0.039090,-0.024873,-0.029805
2083,0.082003,-0.011856,0.030086,-0.033219,0.009262,-0.026075,-0.011445,-0.088163,0.000127,0.063418,...,0.014803,-0.062613,-0.017314,0.059777,-0.045572,0.019472,0.004143,-0.068927,-0.036495,-0.068834
2084,0.138808,-0.021525,0.019962,-0.016783,0.015526,-0.038463,-0.008639,-0.090013,0.012948,0.068980,...,-0.008907,-0.014500,-0.040049,0.066228,-0.036620,0.016215,0.021555,-0.024454,-0.033358,-0.041322
2085,0.135799,0.006806,0.045373,-0.020029,-0.032765,-0.025922,-0.000838,-0.053758,-0.059477,0.061106,...,0.017028,-0.066215,-0.006337,0.075687,-0.051605,0.013428,0.038805,-0.053170,-0.032162,-0.031477


Делаем oversampling с помощью SMOTE:

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smt = SMOTE()
embeddings, levels = smt.fit_resample(df, texts['level'])

In [ ]:
type(levels)


pandas.core.series.Series

In [ ]:
levels = levels.to_frame(name='levels')
levels

,levels
0,1
1,1
2,1
3,1
4,1
...,...
3511,6
3512,6
3513,6
3514,6


In [ ]:
full_data = pd.concat([levels, embeddings], axis=1)
full_data

,levels,0,1,2,3,4,5,6,7,8,...,302,303,304,305,306,307,308,309,310,311
0,1,0.013404,0.081277,-0.002718,0.006781,-0.021074,-0.001101,0.061832,0.013175,0.007073,...,0.053784,0.051911,0.011225,0.030491,-0.012882,0.033562,0.110991,-0.023343,0.023444,-0.097022
1,1,-0.005927,0.059065,-0.040431,-0.006000,-0.020420,-0.034996,0.031260,0.016466,-0.008110,...,0.013393,-0.009853,-0.006460,0.066707,-0.007403,0.025527,0.126925,-0.032500,-0.011747,-0.102165
2,1,0.062544,0.024377,-0.053184,-0.010623,0.000207,-0.063394,0.034824,0.014547,-0.007983,...,0.001290,-0.057028,-0.022071,0.068926,-0.015586,0.045683,0.069069,-0.049771,-0.002949,-0.024824
3,1,0.088431,0.049204,-0.041520,-0.021549,0.002006,-0.061517,0.026090,-0.013590,0.040709,...,-0.014529,-0.058937,-0.037493,0.057122,-0.024982,0.010939,-0.003627,-0.064586,-0.026422,-0.022364
4,1,0.139185,0.043936,-0.022766,0.003944,-0.007901,-0.041158,0.024030,-0.018649,0.038063,...,-0.028922,0.086758,-0.050221,0.048976,-0.052322,-0.012005,-0.079502,-0.043892,0.054148,-0.025723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3511,6,0.068693,-0.003304,-0.005846,-0.009054,-0.043393,-0.041119,-0.042580,-0.062917,-0.000487,...,0.005699,-0.056302,0.002518,0.057438,-0.018762,0.012156,-0.015853,-0.061396,-0.034627,-0.057954
3512,6,0.122930,0.075293,0.033572,-0.012695,0.018148,-0.010031,-0.017042,-0.012174,-0.021309,...,-0.001342,-0.066823,0.028832,-0.013474,-0.011737,0.006608,-0.020308,-0.016444,-0.046831,-0.053896
3513,6,0.080418,0.036435,-0.018094,-0.026967,0.004642,-0.038445,-0.035150,-0.067354,0.024409,...,0.002680,-0.023839,0.020991,0.036845,0.001577,0.003382,-0.026119,-0.035297,-0.029249,-0.024665
3514,6,0.092368,-0.005533,-0.035963,-0.006457,-0.005193,-0.022668,0.014567,-0.058046,-0.027269,...,-0.014897,-0.022501,0.004618,0.034842,0.003790,0.005580,-0.036202,-0.073632,0.024684,-0.059488


In [ ]:
# full_data.to_csv("/content/full_data_tiny.csv", index=True)